# JOIN

This script joins the following datasets, all using pandas:
- Schools
- Wind 
- PM2.5 readings
- Population
- Closest pollution sources for each school

<br>

## Diagram of how the join will work:

![](2022-09-27-18-07-10.png)

Imports

In [1]:
import pandas as pd 
import numpy as np
import os 
import datetime
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as mticker
import plotly.express as px

from netCDF4 import Dataset
# import cartopy.crs as ccrs
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# import dotenv

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

Set folder paths

In [2]:
path_source = 'local'

if path_source == 'gdrive':
  from google.colab import drive
  drive.mount('/content/gdrive')
  gdrive_path = '/content/gdrive/MyDrive/Classes/W210_capstone'
  env_path = '/content/gdrive/MyDrive/.env'
  
elif path_source == 'local':
  gdrive_path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone'
  env_path = '/Users/tj/trevorj@berkeley.edu - Google Drive/MyDrive/.env'

elif path_source == 'work':
  gdrive_path = '/Users/trevorjohnson/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone'
  env_path = '/Users/trevorjohnson/trevorj@berkeley.edu - Google Drive/My Drive/.env'

Read in each dataset

In [3]:
df_census = pd.read_csv(os.path.join(gdrive_path, 'W210_Capstone/Data/census/census_bureau_clean/census_bureau.csv'))
df_wind = pd.read_parquet(os.path.join(gdrive_path, 'W210_Capstone/Data/wind'))
df_pollution = pd.read_csv(os.path.join(gdrive_path, 'W210_Capstone/Data/AirPollution/UW_pm25_zip_monthly_anand_2000-2018-v2.csv'))
df_point_sources = pd.read_csv(os.path.join(gdrive_path, 'W210_Capstone/Data/Point source/pollution_point_sources.csv'))

file_encoding = 'utf8'
with open(os.path.join(gdrive_path, 'JLPS_capstone_project/data/schools_data/filtered_joined_schools_data.csv'), encoding=file_encoding, errors = 'backslashreplace') as my_csv:
  df_schools = pd.read_csv(my_csv, low_memory=False)

pollution point sources assumptions
- For a given year, just find the nearest pollution source for that year. Dont try and track a pollution source across time, and use that same source. 
- We have data every 3 years, assume the following for interpolating the in-between years:
  - 2002 represents 2000 - 2002
  - 2005 represents 2003 - 2005
  - ... 
  - 2017 represents 2015 - 2019

Data Clean:

In [7]:
# clean schools 
df_schools.columns = [i.lower() for i in df_schools.columns]
# only select necessary fields
df_schools = df_schools[['cdscode', 'statustype', 'county', 'street', 'zip_first_five', 'opendate', 'closeddate', 'eilname', 'gsoffered', 
  'latitude', 'longitude', 'lastupdate']]\
  .rename(columns={
    'statustype': 'school_active_status', 'county': 'school_county', 'street': 'school_street', 
    'zip_first_five': 'school_zip', 'opendate': 'school_open_date', 'closeddate': 'school_closed_date', 
    'eilname': 'school_type', 'gsoffered': 'school_grades_offered', 'latitude': 'school_lat', 'longitude': 'school_lon', 
    'lastupdate': 'school_last_updated_date'})

# clean wind
df_wind = df_wind.rename(columns={'lat': 'wind_lat', 'lon': 'wind_lon'})
df_wind['year_month'] = df_wind['year_month'].astype(str).map(lambda x: x[:4] + '-' + x[-2:])
df_wind['year'] = df_wind['year_month'].map(lambda x: int(x[:4]))
df_wind['ZCTA10'] = df_wind['ZCTA10'].astype(int)
df_wind = df_wind[(df_wind['year'] >= 2000) & (df_wind['year'] <= 2019)]

# clean pollution
df_pollution = df_pollution.drop(columns=['Unnamed: 0', 'GEOID10', 'year_month_zip'])

# clean pollution point sources
df_point_sources = df_point_sources.rename(columns={'zip_code': 'point_source_zip'})
df_point_sources['point_source_zip'] = df_point_sources['point_source_zip'].astype(int)
# create an ID field for easier lookups
df_point_sources['point_source_id'] = [i for i in range(df_point_sources.shape[0])]

Join schools, wind, census, and pm2.5 readings

In [8]:
df_all = pd.merge(df_schools, df_wind, left_on = 'school_zip', right_on='ZCTA10', how='left')\
  .merge(df_census, left_on = ['school_zip', 'year'], right_on=['zip', 'year'], how='left')\
  .merge(df_pollution, left_on=['school_zip', 'year_month'], right_on=['ZIP10', 'year_month'], how='left')

Some quality checks

In [206]:
# each school is repeated for every year-month combo. But some schools dont have wind/population data where we dont have that zip code in those datasets. 
yr_mo = df_wind[['year_month']].drop_duplicates().shape[0]
print(f'There are {yr_mo} year month combos')
print('So most schools are repeated 240 times, for the schools that dont have a zip code in the wind data, there are no obs')
df_all['cdscode'].value_counts().to_frame().value_counts('cdscode')

There are 240 year month combos
So most schools are repeated 240 times, for the schools that dont have a zip code in the wind data, there are no obs


cdscode
240    12426
1        871
dtype: int64

## Lat/Lon Join

Join the above dataset to pull the nearest pollution source by year. 

Do so by creating a school <--> source mapping by year

In [207]:
# here is the year mapping since we dont have all years available in the pollution sources. 
# thus, we have to interpolate for the missing years
year_mapping = {
  2000: 2002, 
  2001: 2002, 
  2002: 2002,
  2003: 2005,
  2004: 2005,
  2005: 2005,
  2006: 2008,
  2007: 2008,
  2008: 2008,
  2009: 2011,
  2010: 2011,
  2011: 2011,
  2012: 2014,
  2013: 2014,
  2014: 2014,
  2015: 2017,
  2016: 2017,
  2017: 2017,
  2018: 2017,
  2019: 2017
}

In [210]:
def get_nearest(df_all, df_point_sources, data_year = 2010, partitions = 5, verbose=True):

  df_school_yr = df_all[df_all.year == data_year][['cdscode', 'school_lat', 'school_lon']].drop_duplicates()
  df_ps = df_point_sources[df_point_sources.report_year == year_mapping[data_year]]

  # split data into partitions to avoid overloading memory
  # then loop through each partition and perform the operations
  out_list = []
  for i in range(partitions):
    if verbose:
      print(f'Year: {data_year}. Partition {i+1} of {partitions}')
      
    df_school_yr_i = df_school_yr[df_school_yr['cdscode'] % partitions == i]

    # cross join
    df_school_yr_i['key'] = 0
    df_ps['key'] = 0
    df_cross = pd.merge(df_school_yr_i, df_ps, on = 'key', how='outer')

    # calc distances
    def calc_distance(lat1, lng1, lat2, lng2):
      return ((lat1 - lat2)**2 + (lng1 - lng2)**2)**.5

    df_cross['pollution_school_distance'] = df_cross\
      .apply(lambda df: calc_distance(df['school_lat'], df['school_lon'], df['checked_lat'], df['checked_lon']), axis=1)

    # filter on closest distance per school
    df_closest = df_cross.loc[df_cross.groupby('cdscode').pollution_school_distance.idxmin()]

    # add to list and repeat for other partitions
    out_list.append(df_closest)

  df_out = pd.concat(out_list, ignore_index=True)

  df_out['year'] = data_year
  df_out = df_out.drop(columns=['key'])

  return df_out 

In [184]:
# test on 1 year
df_2000 = get_nearest(df_all, df_point_sources, data_year = 2000, partitions = 5, verbose=True)

Working on partition 1 of 5
Working on partition 2 of 5
Working on partition 3 of 5
Working on partition 4 of 5
Working on partition 5 of 5


## Create School <--> Pollution Source Mapping

Run the function above on all years. 
- This takes about 3.5 - 3.75 min per year
- ended up taking 88 min for all years, w/ 16 gb of ram


In [211]:
%%time 

school_ps_mapping = [get_nearest(df_all, df_point_sources, data_year = i, partitions = 5) for i in range(2000, 2020)]

Year: 2000. Partition 1 of 5
Year: 2000. Partition 2 of 5
Year: 2000. Partition 3 of 5
Year: 2000. Partition 4 of 5
Year: 2000. Partition 5 of 5
Year: 2001. Partition 1 of 5
Year: 2001. Partition 2 of 5
Year: 2001. Partition 3 of 5
Year: 2001. Partition 4 of 5
Year: 2001. Partition 5 of 5
Year: 2002. Partition 1 of 5
Year: 2002. Partition 2 of 5
Year: 2002. Partition 3 of 5
Year: 2002. Partition 4 of 5
Year: 2002. Partition 5 of 5
Year: 2003. Partition 1 of 5
Year: 2003. Partition 2 of 5
Year: 2003. Partition 3 of 5
Year: 2003. Partition 4 of 5
Year: 2003. Partition 5 of 5
Year: 2004. Partition 1 of 5
Year: 2004. Partition 2 of 5
Year: 2004. Partition 3 of 5
Year: 2004. Partition 4 of 5
Year: 2004. Partition 5 of 5
Year: 2005. Partition 1 of 5
Year: 2005. Partition 2 of 5
Year: 2005. Partition 3 of 5
Year: 2005. Partition 4 of 5
Year: 2005. Partition 5 of 5
Year: 2006. Partition 1 of 5
Year: 2006. Partition 2 of 5
Year: 2006. Partition 3 of 5
Year: 2006. Partition 4 of 5
Year: 2006. Pa

In [213]:
# write this mapping table as parquet to disk
school_ps_mapping_df = pd.concat(school_ps_mapping)
fpath = os.path.join(gdrive_path, 'W210_Capstone/Data/school_pollution_mapping/school_pollution_mapping.parquet')
school_ps_mapping_df.to_parquet(fpath)

Join the point sources to schools dataset

In [9]:
# read it back in (optional)
fpath = os.path.join(gdrive_path, 'W210_Capstone/Data/school_pollution_mapping/school_pollution_mapping.parquet')
school_ps_mapping_df = pd.read_parquet(fpath)
# clean up field names and select relevant fields
school_ps_mapping_df = school_ps_mapping_df\
  .rename(columns={'checked_lat': 'pollution_source_lat', 'checked_lon': 'pollution_source_lon', 
    'point_source_zip': 'pollution_source_zip', 'point_source_id': 'pollution_source_id'})

school_ps_mapping_df = school_ps_mapping_df[['cdscode', 'year', 'pollution_source_id', 'pollution_source_lat', 
  'pollution_source_lon', 'PM25_emissions_TPY', 'pollution_school_distance']]

In [10]:
# join
df_all = pd.merge(df_all, school_ps_mapping_df, on=['cdscode', 'year'], how='left')

In [248]:
# save to disk
df_all.to_parquet(os.path.join(gdrive_path, 'W210_Capstone/Data/joined_data/joined_data.parquet'))